In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

In [2]:
N = 100000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

/Users/matthewdrury/Projects/glm/glm.py:198: RuntimeWarning: invalid value encountered in double_scalars
  rel_change = np.abs((dev - dev_prev) / dev_prev)


In [4]:
model.coef_

array([ 0.99362638, -1.99449196,  1.01356462])

## Logistic Model

In [5]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [6]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

/Users/matthewdrury/Projects/glm/glm.py:198: RuntimeWarning: invalid value encountered in double_scalars
  rel_change = np.abs((dev - dev_prev) / dev_prev)


In [7]:
model.coef_

array([ 1.00970884, -2.017974  ,  1.00959388])

## Poission Model

In [8]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [9]:
min(mu)

0.36977162017282833

In [10]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

/Users/matthewdrury/Projects/glm/glm.py:198: RuntimeWarning: invalid value encountered in double_scalars
  rel_change = np.abs((dev - dev_prev) / dev_prev)


In [11]:
model.coef_

array([ 0.99959083, -2.00837388,  1.00360725])